In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 61.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f54b82a2992ce24dbde47bc7182a4f7c0c95a8b1248867e49df5f1fdec142523
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
pip install fsspec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 4.3 MB/s 


In [ ]:
import pyspark as spark
import pandas as pd
import warnings
import os
import glob
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving HISTORICAL_DATA.csv to HISTORICAL_DATA.csv


In [ ]:
BtcDF=pd.read_csv('HISTORICAL_DATA.csv',error_bad_lines=False,engine = 'python',header = None)
Twdf1 = pd.read_csv('1.csv',error_bad_lines=False,engine = 'python',header = None) 
Twdf2=pd.read_csv('2.csv',error_bad_lines=False,engine = 'python',header = None) 
Twdf3=pd.read_csv('3.csv',error_bad_lines=False,engine = 'python',header = None) 

TwdF = pd.concat([Twdf1,Twdf2,Twdf3])

In [ ]:
FullDataTw=sql.createDataFrame(TwdF)
FullDataBtc=sql.createDataFrame(BtcDF)
display(FullDataTw)
display(FullDataBtc)
FullDataTw.show()
FullDataBtc.show()


DataFrame[0: string, 1: string]

DataFrame[0: string, 1: bigint]

+---------+--------------------+
|        0|                   1|
+---------+--------------------+
|2/13/2022|Drake is betting ...|
|2/13/2022|Web3 backers like...|
|2/13/2022|Tom Brady has bee...|
|2/13/2022|Coinbase says it ...|
|2/13/2022|Cryptocurrency pl...|
|2/13/2022|The U.S. seized a...|
|2/13/2022|Autograph, an NFT...|
|2/12/2022|BlockFi is paying...|
|2/12/2022|Today’s crypto ha...|
|2/12/2022|Web3 backers like...|
|2/12/2022|The NBA star and ...|
|2/12/2022|Two crypto exchan...|
|2/12/2022|European football...|
|2/12/2022|Crypto companies ...|
|2/12/2022|If you’re watchin...|
|2/12/2022|John Legend is st...|
|2/12/2022|Opinion | Bitcoin...|
|2/12/2022|Web3 backers like...|
|2/12/2022|A Super Bowl ad c...|
|2/12/2022|U.S. regulators a...|
+---------+--------------------+
only showing top 20 rows

+---------+-----+
|        0|    1|
+---------+-----+
|2/13/2022|42061|
|2/12/2022|42205|
|2/11/2022|42388|
|2/10/2022|43521|
| 2/9/2022|44389|
| 2/8/2022|44038|
| 2/7/2022|43834|
| 

In [ ]:
  FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")
  FullDataTw = FullDataTw.withColumnRenamed('0', 'DateTime') #setting column names of Twitter dataset
  FullDataTw = FullDataTw.withColumnRenamed('1', 'Tweet')
  FullDataBtc = FullDataBtc.withColumnRenamed('0', 'Date_Time') #setting column names of Bitcoin price dataset
  FullDataBtc = FullDataBtc.withColumnRenamed('1', 'Price')
  FullDataBtc = FullDataBtc.filter(FullDataBtc.Date_Time != 'Date') #to get rid of first row with the header

In [ ]:
pip install preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for preprocessor: filename=preprocessor-1.1.3-py3-none-any.whl size=4477 sha256=84f7335e2500786a05c45cf6ce1928e051899ab2ebddea57afb28b0d5cec7615
  Stored in directory: /root/.cache/pip/wheels/0e/b7/36/aa37256db62b4bfd35a6f1b5536e9ba843f257b79dcbf3d5f1
Successfully built preprocessor


In [ ]:
 pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
Tw_samp = FullDataTw.limit(8500) #taking sample of 100 rows and working on it otherwise remove the limit
Tw_samp.show()


+---------+--------------------+
| DateTime|               Tweet|
+---------+--------------------+
|2/13/2022|Drake is betting ...|
|2/13/2022|Web3 backers like...|
|2/13/2022|Tom Brady has bee...|
|2/13/2022|Coinbase says it ...|
|2/13/2022|Cryptocurrency pl...|
|2/13/2022|The U.S. seized a...|
|2/13/2022|Autograph, an NFT...|
|2/12/2022|BlockFi is paying...|
|2/12/2022|Today’s crypto ha...|
|2/12/2022|Web3 backers like...|
|2/12/2022|The NBA star and ...|
|2/12/2022|Two crypto exchan...|
|2/12/2022|European football...|
|2/12/2022|Crypto companies ...|
|2/12/2022|If you’re watchin...|
|2/12/2022|John Legend is st...|
|2/12/2022|Opinion | Bitcoin...|
|2/12/2022|Web3 backers like...|
|2/12/2022|A Super Bowl ad c...|
|2/12/2022|U.S. regulators a...|
+---------+--------------------+
only showing top 20 rows



In [ ]:
import preprocessor as p #cleaning each tweet using tweet-preprocessor like removing hashtags,urls,emojis....
def function_udf(input_str):
    input_str = re.sub(r'RT', '', input_str)
    p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)
    input_str = p.clean(input_str)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", input_str).split())
func_udf = udf(function_udf, StringType())
CleanDF = Tw_samp.withColumn('CleanedTweets', func_udf(Tw_samp['Tweet']))
CleanDF.show()

+---------+--------------------+--------------------+
| DateTime|               Tweet|       CleanedTweets|
+---------+--------------------+--------------------+
|2/13/2022|Drake is betting ...|Drake is betting ...|
|2/13/2022|Web3 backers like...|Web3 backers like...|
|2/13/2022|Tom Brady has bee...|Tom Brady has bee...|
|2/13/2022|Coinbase says it ...|Coinbase says it ...|
|2/13/2022|Cryptocurrency pl...|Cryptocurrency pl...|
|2/13/2022|The U.S. seized a...|The U S seized ab...|
|2/13/2022|Autograph, an NFT...|Autograph an NFT ...|
|2/12/2022|BlockFi is paying...|BlockFi is paying...|
|2/12/2022|Today’s crypto ha...|Todays crypto hat...|
|2/12/2022|Web3 backers like...|Web3 backers like...|
|2/12/2022|The NBA star and ...|The NBA star and ...|
|2/12/2022|Two crypto exchan...|Two crypto exchan...|
|2/12/2022|European football...|European football...|
|2/12/2022|Crypto companies ...|Crypto companies ...|
|2/12/2022|If you’re watchin...|If youre watching...|
|2/12/2022|John Legend is st

In [ ]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.2 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def senti_score_udf(sentence):
    snt = analyser.polarity_scores(sentence)
    return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])
func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['CleanedTweets'])[0])
CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['CleanedTweets'])[1])
CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['CleanedTweets'])[2])
CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['CleanedTweets'])[3])
CleanDF.show()

+---------+--------------------+--------------------+-----+-----+-----+-------+
| DateTime|               Tweet|       CleanedTweets|p_neg|p_neu|p_pos| p_comp|
+---------+--------------------+--------------------+-----+-----+-----+-------+
|2/13/2022|Drake is betting ...|Drake is betting ...|  0.0|0.837|0.163| 0.5994|
|2/13/2022|Web3 backers like...|Web3 backers like...|  0.0|0.898|0.102| 0.3612|
|2/13/2022|Tom Brady has bee...|Tom Brady has bee...|  0.0|0.657|0.343| 0.7003|
|2/13/2022|Coinbase says it ...|Coinbase says it ...|0.098|0.722| 0.18| 0.1531|
|2/13/2022|Cryptocurrency pl...|Cryptocurrency pl...|  0.0|0.777|0.223| 0.5574|
|2/13/2022|The U.S. seized a...|The U S seized ab...|0.138|0.862|  0.0|-0.4939|
|2/13/2022|Autograph, an NFT...|Autograph an NFT ...|  0.0|  1.0|  0.0|    0.0|
|2/12/2022|BlockFi is paying...|BlockFi is paying...|  0.0|  1.0|  0.0|    0.0|
|2/12/2022|Today’s crypto ha...|Todays crypto hat...|0.325|0.675|  0.0|-0.8016|
|2/12/2022|Web3 backers like...|Web3 bac

In [ ]:
FinalTw = CleanDF.selectExpr("DateTime as DateTime", "CleanedTweets as Cleaned_Tweets", "p_neg","p_neu","p_pos","p_comp")
print(FinalTw)
FinalTw.show(5) #selecting necessary columns

DataFrame[DateTime: string, Cleaned_Tweets: string, p_neg: float, p_neu: float, p_pos: float, p_comp: float]
+---------+--------------------+-----+-----+-----+------+
| DateTime|      Cleaned_Tweets|p_neg|p_neu|p_pos|p_comp|
+---------+--------------------+-----+-----+-----+------+
|2/13/2022|Drake is betting ...|  0.0|0.837|0.163|0.5994|
|2/13/2022|Web3 backers like...|  0.0|0.898|0.102|0.3612|
|2/13/2022|Tom Brady has bee...|  0.0|0.657|0.343|0.7003|
|2/13/2022|Coinbase says it ...|0.098|0.722| 0.18|0.1531|
|2/13/2022|Cryptocurrency pl...|  0.0|0.777|0.223|0.5574|
+---------+--------------------+-----+-----+-----+------+
only showing top 5 rows



In [ ]:
FullDataBtc.show(5)

+---------+-----+
|Date_Time|Price|
+---------+-----+
|2/13/2022|42061|
|2/12/2022|42205|
|2/11/2022|42388|
|2/10/2022|43521|
| 2/9/2022|44389|
+---------+-----+
only showing top 5 rows



In [ ]:
# CleandfBtc = FullDataBtc.withColumn("Cleaned_BTC_Time_New",FullDataBtc['Cleaned_BTC_Time'].cast(TimestampType()))
# FinalBtc = FinalBtc.selectExpr("Cleaned_BTC_Time_New as DateTime", "Price")
# FinalBtc = FinalBtc.withColumn("Price",FinalBtc['Price'].cast(DoubleType()))
# FinalBtc.show(10)#In this cell, casting to timesstamp, changing col names and casting price type to double

In [ ]:
FinalTw.printSchema()
FinalTw.count()
FinalTw.show(10)

root
 |-- DateTime: string (nullable = true)
 |-- Cleaned_Tweets: string (nullable = true)
 |-- p_neg: float (nullable = true)
 |-- p_neu: float (nullable = true)
 |-- p_pos: float (nullable = true)
 |-- p_comp: float (nullable = true)

+---------+--------------------+-----+-----+-----+-------+
| DateTime|      Cleaned_Tweets|p_neg|p_neu|p_pos| p_comp|
+---------+--------------------+-----+-----+-----+-------+
|2/13/2022|Drake is betting ...|  0.0|0.837|0.163| 0.5994|
|2/13/2022|Web3 backers like...|  0.0|0.898|0.102| 0.3612|
|2/13/2022|Tom Brady has bee...|  0.0|0.657|0.343| 0.7003|
|2/13/2022|Coinbase says it ...|0.098|0.722| 0.18| 0.1531|
|2/13/2022|Cryptocurrency pl...|  0.0|0.777|0.223| 0.5574|
|2/13/2022|The U S seized ab...|0.138|0.862|  0.0|-0.4939|
|2/13/2022|Autograph an NFT ...|  0.0|  1.0|  0.0|    0.0|
|2/12/2022|BlockFi is paying...|  0.0|  1.0|  0.0|    0.0|
|2/12/2022|Todays crypto hat...|0.325|0.675|  0.0|-0.8016|
|2/12/2022|Web3 backers like...|  0.0|0.898|0.102| 0.36

In [ ]:
FullDataBtc.printSchema()
FullDataBtc.count()

root
 |-- Date_Time: string (nullable = true)
 |-- Price: long (nullable = true)



1556

In [ ]:
# dt_truncated = ((round(unix_timestamp(col('DateTime')) / 3600) * 3600).cast('timestamp'))
# print(dt_truncated)
# FinalTw = FinalTw.withColumn('dt_truncated', dt_truncated)
# FinalTw = FinalTw.selectExpr("dt_truncated as Date_Time","Cleaned_Tweets","p_neg","p_neu","p_pos","p_comp")
# UTC = ((unix_timestamp(col('Date_Time'))+ 5*60*60).cast('timestamp'))
# FinalTw = FinalTw.withColumn('UTC', UTC)
# FinalTw = FinalTw.selectExpr("UTC as Date_Time","Cleaned_Tweets","p_neg","p_neu","p_pos","p_comp")
# FinalTw.show(10)

In [ ]:
FinalTw.registerTempTable("temp")
FinalTw_avg = sql.sql("SELECT DateTime As DateTime,AVG(p_neg) as P_Neg,AVG(p_neu) as P_Neu,AVG(p_pos) as P_Pos,AVG(p_comp) as P_Comp FROM temp GROUP BY DateTime")
print(FinalTw_avg)
#FinalTw_avg = FinalTw.select("DateTime","polarity","subj","p_pos","p_neg").groupBy("DateTime").agg(avg(col("polarity","subj","p_pos","p_neg")))
FinalTw_avg.show(1)

DataFrame[DateTime: string, P_Neg: double, P_Neu: double, P_Pos: double, P_Comp: double]
+--------+-------------------+-----------------+-------------------+------------------+
|DateTime|              P_Neg|            P_Neu|              P_Pos|            P_Comp|
+--------+-------------------+-----------------+-------------------+------------------+
|1/4/2022|0.03385714299622036|0.886952383177621|0.07919047666447503|0.1046476172549384|
+--------+-------------------+-----------------+-------------------+------------------+
only showing top 1 row



In [ ]:
from pyspark.sql import functions as f
df_with_text = FinalTw.groupby("DateTime").agg(f.concat_ws(" ", f.collect_list(FinalTw.Cleaned_Tweets)))
df_with_text.show(10)

+---------+------------------------------------------+
| DateTime|concat_ws( , collect_list(Cleaned_Tweets))|
+---------+------------------------------------------+
| 1/1/2020|                      The trend is your...|
| 1/1/2022|                      Ten of twelve Bit...|
|1/10/2020|                      Bitcoin Beginners...|
|1/10/2022|                      THE COIN BEHIND T...|
|1/11/2019|                      My latest work co...|
|1/11/2020|                        Who are the mos...|
|1/11/2021|                      Bitcoin at 10 000...|
|1/11/2022|                      Got USD bitcoin m...|
|1/12/2019|                      Bitcoin destroys ...|
|1/12/2020|                      The Bitcoin netwo...|
+---------+------------------------------------------+
only showing top 10 rows



In [ ]:
FinalTw_avg.count()
from pyspark.sql.functions import *
df_sort = FinalTw_avg.sort(asc("DateTime"))
df_sort.show(1)

+--------+-----+------------------+--------------------+-------------------+
|DateTime|P_Neg|             P_Neu|               P_Pos|             P_Comp|
+--------+-----+------------------+--------------------+-------------------+
|1/1/2020|  0.0|0.9449999928474426|0.054999999701976776|0.24695000052452087|
+--------+-----+------------------+--------------------+-------------------+
only showing top 1 row



In [ ]:
FinalTw_avg.registerTempTable("avgs")
FullDataBtc.registerTempTable("prices")
results = sql.sql("SELECT DateTime, P_Neg, P_Neu, P_Pos, P_Comp, Price FROM avgs JOIN prices ON avgs.DateTime = prices.Date_Time order by avgs.DateTime")
#results = results.selectExpr("DateTime","avg(polarity)","avg(subj)","avg(p_pos)","avg(p_neg)","Price") #Use this line if you are using text blob package
results.show()

+---------+--------------------+------------------+--------------------+--------------------+-----+
| DateTime|               P_Neg|             P_Neu|               P_Pos|              P_Comp|Price|
+---------+--------------------+------------------+--------------------+--------------------+-----+
| 1/1/2020|                 0.0|0.9449999928474426|0.054999999701976776| 0.24695000052452087| 7200|
| 1/1/2022| 0.08234999924898148| 0.785499992966652| 0.13220000118017197| 0.11552499942481517|47738|
|1/10/2020|0.026937499176710844|0.7418125011026859| 0.16875000018626451| 0.30407499615103006| 8187|
|1/10/2022|0.017937499564141035|0.8960624970495701| 0.08600000059232116| 0.20977499801665545|41832|
|1/11/2019|                 0.0|0.8600000143051147| 0.14000000059604645| 0.38179999589920044| 3617|
|1/11/2020|                 0.0|           0.40625|             0.09375| 0.18410000205039978| 8024|
|1/11/2021|                 0.0|               1.0|                 0.0|                 0.0|35544|


In [ ]:
results.repartition(1).write.csv("DataforModelExec.csv") #this will write df to single csv instead of writing diff csv acc to partitions 